# Creating dataset to analyze bike sharing demand in Washington D.C.

In [33]:
import pandas as pd
import glob
from datetime import datetime


In [34]:
# display all rows and columns in the dataframes
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)


## Bike sharing demand dataset

In [35]:
# merge the bike sharing demand csv files into one dataframe for Washington, D.C.
path = r'C:\Users\julia\Documents\Studium\Master\2. Semester\Master-Seminar Applied Econometrics - Data Science Basics\bike-sharing\datasets\bike-sharing'
all_files = glob.glob(path + "/*.csv")

df_list = []

for file in all_files:
    df_comp = pd.read_csv(file)
    df_list.append(df_comp)
    
df_bike = pd.concat(df_list, axis=0, ignore_index=True)


In [36]:
# get overview of dataframe
print(df_bike.shape)
df_bike.head()


(22544730, 9)


,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,5th & F St NW,31620,5th & F St NW,W00247,Member
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,14th & Harvard St NW,31101,14th & V St NW,W00675,Casual
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,Georgia & New Hampshire Ave NW,31104,Adams Mill & Columbia Rd NW,W00357,Member
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,10th & U St NW,31503,Florida Ave & R St NW,W00970,Member
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,Adams Mill & Columbia Rd NW,31106,Calvert & Biltmore St NW,W00346,Casual


In [37]:
# drop columns that are irrelevant
df_bike_drop = df_bike.drop(['Duration', 'Start station number', 'Start station', 'End station number', 
                             'End station', 'Bike number', 'End date'], axis=1)

# create new column that only contains the date
df_bike_drop['date'] = df_bike_drop['Start date'].apply(lambda x: x[:10])

# drop the Start date column
df_bike_drop.drop(['Start date'], axis=1, inplace=True)




In [38]:
# create two new columns for number of casual and registered customers per day
df_bike_drop['casual'] = df_bike_drop['Member type'].apply(lambda x: 1 if x == 'Casual' else 0)
df_bike_drop['registered'] = df_bike_drop['Member type'].apply(lambda x: 1 if x == 'Member' else 0)

# create new dataframe with date, casual, registered and total customers
cust_list = ['casual', 'registered']
df_bike_list = []

for cust in cust_list:
    series_bike_users = df_bike_drop.groupby('date')[cust].sum()
    df_bike_users = series_bike_users.to_frame()
    df_bike_list.append(df_bike_users)
    
# concat both dataframes saved in list and add new column containing total number of customers
df_bike_cust = pd.concat(df_bike_list, axis=1)
df_bike_cust.reset_index(inplace=True)
df_bike_cust['total_cust'] = df_bike_cust['casual'] + df_bike_cust['registered']
df_bike_cust



,date,casual,registered,total_cust
0,2011-01-01,330,629,959
1,2011-01-02,130,651,781
2,2011-01-03,120,1181,1301
3,2011-01-04,107,1429,1536
4,2011-01-05,82,1489,1571
5,2011-01-06,88,1485,1573
6,2011-01-07,148,1345,1493
7,2011-01-08,68,871,939
8,2011-01-09,54,748,802
9,2011-01-10,41,1257,1298


## Holiday dataset

In [39]:
# manual creation of dataframe containing information on holidays in Washington, D.C.
# source for this is https://dchr.dc.gov/page/holiday-schedules-2018, https://dchr.dc.gov/page/holiday-schedules-2016-and-2017
# https://dchr.dc.gov/page/holiday-schedules-2014-and-2015, https://dchr.dc.gov/page/holiday-schedules-2012-and-2013
# https://dchr.dc.gov/page/holiday-schedules-2010-and-2011

df_holiday = pd.DataFrame(columns=['date', 'holiday'])

dates = ['2018-01-01', '2018-01-15', '2018-02-19', '2018-04-16',
         '2018-05-28', '2018-07-04', '2018-09-03', '2018-10-08',
         '2018-11-12', '2018-11-22', '2018-12-25', '2017-01-02',
         '2017-01-16', '2017-01-20', '2017-02-20', '2017-04-17',
         '2017-05-29', '2017-07-04', '2017-09-04', '2017-10-09',
         '2017-11-10', '2017-11-23', '2017-12-25', '2016-01-01',
         '2016-01-18', '2016-02-15', '2016-04-15', '2016-05-30',
         '2016-07-04', '2016-09-05', '2016-10-10', '2016-11-11',
         '2016-11-24', '2016-12-26', '2015-01-01', '2015-01-19',
         '2015-02-16', '2015-04-16', '2015-05-25', '2015-07-03',
         '2015-09-07', '2015-10-12', '2015-11-11', '2015-11-26',
         '2015-12-25', '2014-01-01', '2014-01-20', '2014-02-17',
         '2014-04-16', '2014-05-26', '2014-07-04', '2014-09-01', 
         '2014-10-13', '2014-11-11', '2014-11-27', '2014-12-25',
         '2013-01-01', '2013-01-21', '2013-01-20', '2013-02-18', 
         '2013-04-16', '2013-05-27', '2013-07-04', '2013-09-02',
         '2013-10-14', '2013-11-11', '2013-11-28', '2013-12-25',
         '2012-01-02', '2012-01-16', '2012-02-20', '2012-04-16',
         '2012-05-28', '2012-07-04', '2012-09-03', '2012-10-08',
         '2012-11-12', '2012-11-22', '2012-12-25', '2011-01-17',
         '2011-02-21', '2011-04-15', '2011-05-30', '2011-07-04',
         '2011-09-05', '2011-10-10', '2011-11-11', '2011-11-24',
         '2011-12-26']

df_holiday['date'] = dates
df_holiday['holiday'] = 1
df_holiday.head()


,date,holiday
0,2018-01-01,1
1,2018-01-15,1
2,2018-02-19,1
3,2018-04-16,1
4,2018-05-28,1


## Weather dataset

In [43]:
# read in the weather data for Washington, D.C.
path = r'C:\Users\julia\Documents\Studium\Master\2. Semester\Master-Seminar Applied Econometrics - Data Science Basics\bike-sharing\datasets\weather-data'
all_files = glob.glob(path + "/*.csv")

df_list_weather = []

for file in all_files:
    df_comp_weather = pd.read_csv(file)
    df_list_weather.append(df_comp_weather)
    
df_weather = pd.concat(df_list_weather, axis=0, ignore_index=True)


C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7,11,21,27,29,31,35,37,39,41,43,45,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7,21,27,31,37,39,41,43,45,47,49,51,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7,11,19,21,27,31,33,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7,11,13,15,17,19,21,23,25,27,29,31,33,35) have mixed types

In [44]:
# number of rows and columns in weather dataframe
df_weather.shape


(268114, 54)

In [45]:
df_weather.describe()

,AWND,ELEVATION,LATITUDE,LONGITUDE,PRCP,TAVG,TMAX,TMIN,TOBS,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11,WT13,WT14,WT15,WT16,WT17,WT18,WT19,WT21,WT22
count,11506.000000,268114.000000,268114.000000,268114.000000,263107.000000,6150.000000,42469.000000,42456.000000,31298.000000,3911.0,328.0,1959.0,295.0,58.0,266.0,985.0,12.0,346.0,800.0,229.0,9.0,1032.0,9.0,178.0,1.0,41.0,10.0
mean,3.166974,81.557438,38.971833,-76.989676,3.721575,14.312520,18.989263,8.445051,11.000150,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,1.510937,49.302938,0.194212,0.252576,10.440575,9.635069,10.127129,9.679169,9.750956,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0
min,0.200000,0.000000,38.467400,-77.497598,0.000000,-12.900000,-11.700000,-20.000000,-18.300000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,2.100000,43.900000,38.847200,-77.183060,0.000000,6.500000,10.600000,0.600000,2.800000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,2.900000,83.200000,38.991000,-77.010883,0.000000,15.300000,20.000000,8.300000,11.700000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,4.000000,115.800000,39.114700,-76.784300,2.000000,23.000000,27.800000,17.200000,19.400000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,14.500000,248.100000,39.339729,-76.488006,252.500000,32.800000,41.100000,29.400000,33.900000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [28]:
# get an idea of the datatypes on the weather dataframe
df_weather.dtypes


AWND               float64
AWND_ATTRIBUTES     object
DATE                object
ELEVATION          float64
LATITUDE           float64
LONGITUDE          float64
NAME                object
PRCP               float64
PRCP_ATTRIBUTES     object
STATION             object
TAVG               float64
TAVG_ATTRIBUTES     object
WT01               float64
WT01_ATTRIBUTES     object
WT02               float64
WT02_ATTRIBUTES     object
WT03               float64
WT03_ATTRIBUTES     object
WT04               float64
WT04_ATTRIBUTES     object
WT05               float64
WT05_ATTRIBUTES     object
WT06               float64
WT06_ATTRIBUTES     object
WT08               float64
WT08_ATTRIBUTES     object
WT09               float64
WT09_ATTRIBUTES     object
WT11               float64
WT11_ATTRIBUTES     object
WT13               float64
WT13_ATTRIBUTES     object
WT14               float64
WT14_ATTRIBUTES     object
WT15               float64
WT15_ATTRIBUTES     object
WT16               float64
W

In [29]:
df_weather.describe(include='all')

,AWND,AWND_ATTRIBUTES,DATE,ELEVATION,LATITUDE,LONGITUDE,NAME,PRCP,PRCP_ATTRIBUTES,STATION,TAVG,TAVG_ATTRIBUTES,WT01,WT01_ATTRIBUTES,WT02,WT02_ATTRIBUTES,WT03,WT03_ATTRIBUTES,WT04,WT04_ATTRIBUTES,WT05,WT05_ATTRIBUTES,WT06,WT06_ATTRIBUTES,WT08,WT08_ATTRIBUTES,WT09,WT09_ATTRIBUTES,WT11,WT11_ATTRIBUTES,WT13,WT13_ATTRIBUTES,WT14,WT14_ATTRIBUTES,WT15,WT15_ATTRIBUTES,WT16,WT16_ATTRIBUTES,WT17,WT17_ATTRIBUTES,WT18,WT18_ATTRIBUTES,WT19,WT19_ATTRIBUTES,WT21,WT21_ATTRIBUTES,WT22,WT22_ATTRIBUTES
count,11506.000000,11506,268114,268114.000000,268114.000000,268114.000000,268114,263107.000000,263107,268114,6150.000000,6150,3911.0,3911,328.0,328,1959.0,1959,295.0,295,58.0,58,266.0,266,985.0,985,12.0,12,346.0,346,800.0,800,229.0,229,9.0,9,1032.0,1032,9.0,9,178.0,178,1.0,1,41.0,41,10.0,10
unique,NaN,3,2922,NaN,NaN,NaN,222,NaN,44,223,NaN,1,NaN,4,NaN,2,NaN,4,NaN,5,NaN,4,NaN,3,NaN,2,NaN,1,NaN,3,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1
top,NaN,",,W",2017-07-29,NaN,NaN,NaN,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",NaN,",,N",USW00093721,NaN,"H,,S",NaN,",,W",NaN,",,W",NaN,",,7",NaN,",,7",NaN,",,7",NaN,",,7",NaN,",,W",NaN,",,W",NaN,",,7",NaN,",,X",NaN,",,X",NaN,",,X",NaN,",,X",NaN,",,X",NaN,",,X",NaN,",,X",NaN,",,X",NaN,",,X"
freq,NaN,10776,118,NaN,NaN,NaN,2922,NaN,206670,2922,NaN,6150,NaN,2992,NaN,240,NaN,1026,NaN,159,NaN,29,NaN,207,NaN,797,NaN,12,NaN,295,NaN,800,NaN,229,NaN,9,NaN,1032,NaN,9,NaN,178,NaN,1,NaN,41,NaN,10
mean,3.166974,NaN,NaN,81.557438,38.971833,-76.989676,NaN,3.721575,NaN,NaN,14.312520,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN
std,1.510937,NaN,NaN,49.302938,0.194212,0.252576,NaN,10.440575,NaN,NaN,9.635069,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN
min,0.200000,NaN,NaN,0.000000,38.467400,-77.497598,NaN,0.000000,NaN,NaN,-12.900000,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN
25%,2.100000,NaN,NaN,43.900000,38.847200,-77.183060,NaN,0.000000,NaN,NaN,6.500000,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN
50%,2.900000,NaN,NaN,83.200000,38.991000,-77.010883,NaN,0.000000,NaN,NaN,15.300000,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN
75%,4.000000,NaN,NaN,115.800000,39.114700,-76.784300,NaN,2.000000,NaN,NaN,23.000000,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN


In [30]:
# show all available weather situation columns and create a list without the attribute columns
weathersit_list = list(df_weather.columns)[12:]
weathersit = [x for x in weathersit_list if len(x) < 5]
weathersit


['WT01',
 'WT02',
 'WT03',
 'WT04',
 'WT05',
 'WT06',
 'WT08',
 'WT09',
 'WT11',
 'WT13',
 'WT14',
 'WT15',
 'WT16',
 'WT17',
 'WT18',
 'WT19',
 'WT21',
 'WT22']

In [31]:
# create a new data frame that contains only average temperature, windspeed, precipitation and weathersituation
# per day across all stations in Washington, D.C.
var_list = ['TAVG', 'PRCP', 'AWND', weathersit]
df_avg_list = []

for var in var_list:
    if type(var) != list:
        data_avg_vars = df_weather.groupby('DATE')[var].mean()
        df_avg_vars = data_avg_vars.to_frame()
        df_avg_list.append(df_avg_vars)
    else:
        for sit in weathersit:
            data_wsit = df_weather.groupby('DATE')[sit].max()
            df_wsit = data_wsit.to_frame()
            df_avg_list.append(df_wsit)
            
df_weather_aggr = pd.concat(df_avg_list, axis=1)
df_weather_aggr


,TAVG,PRCP,AWND,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11,WT13,WT14,WT15,WT16,WT17,WT18,WT19,WT21,WT22
DATE,,,,,,,,,,,,,,,,,,,,,
2011-01-01,NaN,0.069333,2.575000,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2011-01-02,NaN,1.037349,3.925000,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2011-01-03,NaN,1.878824,3.625000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-04,NaN,0.000000,1.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-05,NaN,0.000000,2.950000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-06,NaN,0.000000,1.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
2011-01-07,NaN,0.101205,2.550000,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
2011-01-08,NaN,0.285057,4.575000,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
2011-01-09,NaN,0.283529,6.425000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# reset index and change column names
df_weather_aggr.reset_index(inplace=True)

df_weather_aggr = df_weather_aggr.rename(columns={'DATE': 'date', 'TAVG': 'temp', 'PRCP': 'precip', 'AWND': 'wind',
                        'WT01': 'wt_fog', 'WT02': 'wt_heavy_fog', 'WT03': 'wt_thunder', 'WT04': 'wt_sleet', 
                        'WT05': 'wt_hail', 'WT06': 'wt_glaze', 'WT08':'wt_haze', 'WT09':'wt_drift_snow',
                        'WT11': 'wt_high_wind'})
df_weather_aggr.head()


,date,temp,precip,wind,wt_fog,wt_heavy_fog,wt_thunder,wt_sleet,wt_hail,wt_glaze,wt_haze,wt_drift_snow,wt_high_wind
0,2015-01-01,-0.733333,0.000000,2.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-02,2.733333,0.000000,1.200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-03,1.633333,2.548101,1.375,1.0,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN
3,2015-01-04,8.600000,15.772619,4.750,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,2015-01-05,5.833333,2.194048,6.375,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Combination of all three separate datasets